# 05: Bagging and random forests

In [13]:
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

import mylib as my

The goal of ensemble methods to to reduce bias and/or variance help prevent overfitting. In this notebook we look at two ensemble methods: bagging and random forests.

## Bootstrap samples
Let's start by seeing how we can draw a bootstrap sample given a dataset $D$. A bootstrap sample is a sample drawn randomly with replacement from the given dataset such that the size of the sample is the same as the size of the original dataset. That means some examples will show up multiple times in the drawn sample.

In the example below, we are using a subset of the car dataset with classes indicating whether the car is in acceptable or unacceptable condition. The description of the original car dataset can be found at [this page](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation).

In [2]:
df = pd.read_csv('datasets/ua_car.csv')
ds = my.DataSet(df, y=True)
print(df.iloc[:,-1].value_counts())

unacc    384
acc      384
Name: y, dtype: int64


In [3]:
train, test = ds.train_test_split(test_portion=.25, shuffle=True)
print(train)
print(test)

    buying maintenance  doors persons luggage safety      y
27   vhigh       vhigh  5more       4     big   high  unacc
609    med        high      3       4     big    med    acc
744    low        high  5more    more   small    med    acc
714    low       vhigh  5more    more     big   high    acc
156  vhigh         med      4       2   small    med  unacc
..     ...         ...    ...     ...     ...    ...    ...
540   high         low      3    more     med    med    acc
300    med        high      2       2     big   high  unacc
194  vhigh       vhigh      2    more   small   high  unacc
158    med        high  5more       2   small   high  unacc
208    med         low      2       2     big   high  unacc

[576 rows x 7 columns]
    buying maintenance  doors persons luggage safety      y
319    low         low  5more       4     med    low  unacc
69    high         med      4    more   small    low  unacc
46     med         low      2    more     med    low  unacc
161    med      

Given the above training set, we can draw a bootstrap sample like this:

In [4]:
sample_indexes = np.random.randint(0, train.N, size=train.N)
bootstrap_sample = train.examples.iloc[sample_indexes, :]
bootstrap_sample

,buying,maintenance,doors,persons,luggage,safety,y
341,high,vhigh,2,4,med,high,unacc
49,low,high,4,4,small,low,unacc
58,low,med,2,more,small,low,unacc
75,high,high,3,2,small,low,unacc
654,med,med,4,4,small,high,acc
...,...,...,...,...,...,...,...
251,low,high,4,2,med,high,unacc
370,vhigh,low,3,2,med,med,unacc
119,high,vhigh,5more,4,small,low,unacc
415,vhigh,med,5more,more,small,high,acc


of which:

In [5]:
print("{:.2%}".format(
    pd.unique(bootstrap_sample.index).shape[0] / len(bootstrap_sample)), 'are unique examples')
print("{:.2%}".format(
    1 - pd.unique(bootstrap_sample.index).shape[0] / len(bootstrap_sample)), 'are repeated examples')

64.93% are unique examples
35.07% are repeated examples


Sometimes, it's useful to be able to identify the examples that are included in a given sample and those that aren't. Here are two functions for doing so.

In [6]:
def examples_in_sample(examples, sample):
    return examples[examples.index.isin(sample.index)]

def examples_not_in_sample(examples, sample):
    return examples[~examples.index.isin(sample.index)]

Here are the examples from the training set what are in the above bootstrap sample:

In [7]:
examples_in_sample(train.examples, bootstrap_sample)

,buying,maintenance,doors,persons,luggage,safety,y
609,med,high,3,4,big,med,acc
156,vhigh,med,4,2,small,med,unacc
254,med,med,4,2,small,high,unacc
385,vhigh,med,2,4,med,high,acc
547,high,low,4,4,big,med,acc
...,...,...,...,...,...,...,...
3,high,vhigh,3,more,med,high,unacc
189,vhigh,low,4,4,small,low,unacc
328,med,vhigh,3,4,med,low,unacc
540,high,low,3,more,med,med,acc


And here are the examples from the training set that are not in the above bootstrap sample:

In [8]:
examples_not_in_sample(train.examples, bootstrap_sample)

,buying,maintenance,doors,persons,luggage,safety,y
27,vhigh,vhigh,5more,4,big,high,unacc
744,low,high,5more,more,small,med,acc
714,low,vhigh,5more,more,big,high,acc
381,high,vhigh,2,2,big,high,unacc
5,med,low,5more,4,small,low,unacc
...,...,...,...,...,...,...,...
345,vhigh,high,3,more,small,high,unacc
71,low,med,2,2,med,low,unacc
300,med,high,2,2,big,high,unacc
194,vhigh,vhigh,2,more,small,high,unacc


## Bagging
The simplest form of ensemble methods is called **bagging** which stands for **bootstrap aggregation**. The idea is simple:
* take $T$ bootstrap samples from the given dataset
* for each bootstrap sample, train a decision tree DT
* the predicted label of an unseen example is the average(for regression problems) or the plurality vote (for classification problems) of all the output predicted by all the trained $T$ trees.

Here is a simple implementation of bagging.

In [10]:
class Bagger:
    def __init__(self, dataset, nTrees):
        self.ds = dataset
        self.nTrees = nTrees
        self.classifiers = []
        self.samples = []
        self.make_trees()

    def make_trees(self):
        indexes = np.random.randint(0, self.ds.N,(self.ds.N,self.nTrees))
        for i in range(self.nTrees):
            # Create bootstrap samples one for each tree
            self.samples.append(self.ds.examples.iloc[indexes[:, i], :])

            # Build classifiers
            self.classifiers.append(my.DecisionTreeClassifier(my.DataSet(self.samples[i])))

    def predict(self, unseen):
        """
        Returns the most probable label (or class) for each unseen input. The
        unseen needs to be a data series with the same features (as indexes) as the 
        training data. It can also be a data frame with the same features as 
        the training data.
        """
        if unseen.ndim == 1:
            classes = np.array([ dt.predict(unseen) for dt in self.classifiers ])
            classes = classes[classes != None]
            return st.mode(classes).mode[0]
        
        else:
            return np.array([self.predict(unseen.iloc[i,:]) for i in range(len(unseen))]) 

## Random forests
Bagging is not exclusive to decision trees; it can be used with other models. Random forests is bagging applied exclusively to decision trees. In addition to obtaining $T$ random bootstrap samples, it also requires what is sometimes called **feature bagging**. Feature bagging requires that only a randomly selected subset of the features is considered at each node during the construction of the decision tree. 

That means we need to modify our implementation of the decision tree such that it takes a numeric parameter named `nFeatures` which defaults to 0. If `nFeatures` is 0, then the tree functions as normal. If not, it picks this many features randomly and only consider the best of those during the construction of the tree. The provided `my.DecisionTreeClassifier` class already has these changes.

For prediction, a plurality vote of the $T$ predicted labels is returned. Here is a simple implementing of random forests. Think about the similarities and differences between these too classes.

In [11]:
class RandomForest:
    def __init__(self, dataset, nTrees, nFeatures=0):
        self.ds = dataset
        self.nTrees = nTrees
        self.nFeatures = nFeatures
        self.classifiers = []
        self.samples = []
        self.make_forest()

    def make_forest(self):
        indexes = np.random.randint(0, self.ds.N,(self.ds.N,self.nTrees))
        for i in range(self.nTrees):
            # Create bootstrap samples one for each tree
            self.samples.append(self.ds.examples.iloc[indexes[:, i], :])

            # Build classifiers
            self.classifiers.append(my.DecisionTreeClassifier(my.DataSet(self.samples[i]), nFeatures=self.nFeatures))

    def predict(self, unseen):
        """
        Returns the most probable label (or class) for each unseen input. The
        unseen needs to be a data series with the same features (as indexes) as the 
        training data. It can also be a data frame with the same features as 
        the training data.
        """
        if unseen.ndim == 1:
            classes = np.array([ dt.predict(unseen) for dt in self.classifiers ])
            classes = classes[classes != None]
            return st.mode(classes).mode[0]
        
        else:
            return np.array([self.predict(unseen.iloc[i,:]) for i in range(len(unseen))]) 

## Testing

In [12]:
dt = my.DecisionTreeClassifier(train)
cm = my.confusion_matrix(test.target, dt.predict(test.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)

print(cm)
print('Decistion tree accuracy: ', accuracy)


bg = Bagger(train, 20)
cm = my.confusion_matrix(test.target, bg.predict(test.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)

print(cm)
print('Bagger accuracy: ', accuracy)

rf = RandomForest(train, 20, nFeatures=3)
cm = my.confusion_matrix(test.target, rf.predict(test.examples.iloc[:,:-1]))
accuracy = np.trace(cm) / np.sum(cm)

print(cm)
print('Random forests accuracy: ', accuracy)

[[91  2]
 [ 5 94]]
Decistion tree accuracy:  0.9635416666666666
[[93  0]
 [ 6 93]]
Bagger accuracy:  0.96875
[[92  1]
 [ 6 93]]
Random forests accuracy:  0.9635416666666666


You should try different values for `nTrees` and `nFeatures`. These variables are considered hyperparameters, and cross-validation can be used to determine the best values for them. Common values for `nFeatures` are $\sqrt{m}$ and $log_2(m)$ where $m$ is the number of features.

## Out of bag score
Another way of testing random forests is to calculate the so-called **out-of-bag** score. Such a score does not require splitting the dataset into a training and test sets. One way to calculate it is to identify for each example $x$ in the dataset the list of trees that are trained using samples that do not include $x$; let's call this list of trees $D_x$. We then call the `predict` method on each tree of $D_x$ to get the list of predicted classes for example $x$; let's call this list of classes $C_x$. Finally we find the class in $C_x$ that repeats the most and report it as the predicted class of $x$; let's call it $h_x$.

Doing this for each example in the dataset gives us an array of predicted classes, which we can compare against the actual target classes of these examples. Using the confusion matrix we can report the accuracy as the out of bag score.

Notice that the above implementations of `Bagger` and `RandomForest` already give you access to the bootstrap samples and the classifiers that are trained on them. You can use that to find out what sample does not include a given example.

## CHALLENGE
Write a function that calculates the out of bag score as described above given three arguments: a dataset, number of trees (`nTrees`), and number of features (`nFeatures`). The function should use these arguments to create a random forest object to use for calculating this score.

Test and report the out of bag scores for the whole car dataset and for when `nTrees` is 10, 15, and 20.

In [180]:
def OutofBag(dataset, nTrees, nFeatures = 0):
    Dx = []
    Hx = []
    predictClass = []
    targetClass = []
    
    #Instantiate Random Forest
    rf = RandomForest(dataset, nTrees, nFeatures)
    
    #examples_not_in_sample(examples, sample)
    
    #for each sample in the samples array, get a list of trained trees that don't include the example
    for sample in rf.samples:
        Dx.append(examples_not_in_sample(dataset.examples, sample))


    #////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    
    #for tree in Dx:
         #   Cx.append(randForest.predict(tree))
    
    #We will be predicting a class with trees it wasn't used in
    #We check if each example is used in a given tree, and store it in tempCx
    for i in range(len(dataset.examples)):
        tempCx = []
        
        #loop through the number of trees: 10, 15, 20 in this case
        for j in range (nTrees):
            #if it matches the example not in sample, append it to the temporary array
            if i in Dx[j].index:
                #tempCx.append(randForest.classifiers[j].predict(Dx[i]))
                tempCx.append(rf.classifiers[j].predict(dataset.examples.iloc[i]))
      
    
         # for clsList in Cx:
        #    for i in range(len(clsList)):
         #       find max unique value
    
        #append 'NoData' if empty
        if len(tempCx) > 0:
            val, cnts = np.unique(tempCx, return_counts = True)
            #Get the maximum predicted class
            maxPrediction = val[cnts.argmax()]
            #Store it permanently in Hx
            Hx.append(maxPrediction)
        else:
            Hx.append('NoData')

        #End of Cx -> Hx For Loop
        #/////////////////////////////////////////////////////////////////////////////////////////////////////////
            
    #for each predicted class in Hx, append it to the predictClass array and append its actual targeted class
    # counterpart to the targetClass array
    for predicted in range(len(Hx)):
        if Hx[predicted] != 'NoData':
            predictClass.append(Hx[predicted])
            targetClass.append(dataset.target[predicted])
    
    #Do your confusion matrix and confusion matrix accuracy
    
    cm = my.confusion_matrix(np.array(targetClass), np.array(predictClass))
    accuracy = np.trace(cm) / np.sum(cm)   
    
    return accuracy

In [181]:
car10 = OutofBag(ds, 10)

print('10 trees accuracy: \n', car10)

car15 = OutofBag(ds, 15)

print('\n15 trees accuracy: \n', car15)

car20 = OutofBag(ds, 20)

print('\n20 trees accuracy: \n', car20)

10 trees accuracy: 
 0.9526315789473684

15 trees accuracy: 
 0.9505208333333334

20 trees accuracy: 
 0.9557291666666666
